In [ ]:
!pip install quantdsl

In [ ]:
%%writefile powerplant.py
from quantdsl.semantics import Choice, TimeDelta, Wait, inline


def PowerPlant(start, end, temp):
    if (start < end):
        Wait(start, Choice(
            PowerPlant(Tomorrow(start), end, Hot()) + ProfitFromRunning(start, temp),
            PowerPlant(Tomorrow(start), end, Stopped(temp))
        ))
    else:
        return 0


@inline
def Power(start):
    DayAhead(start, 'POWER')


@inline
def Gas(start):
    DayAhead(start, 'GAS')


@inline
def DayAhead(start, name):
    ForwardMarket(Tomorrow(start), name)

    
@inline
def Tomorrow(start):
    start + TimeDelta('1d')


@inline
def ProfitFromRunning(start, temp):
    if temp == Cold():
        return 0.3 * Power(start) - Gas(start)
    elif temp == Warm():
        return 0.6 * Power(start) - Gas(start)
    else:
        return Power(start) - Gas(start)


@inline
def Stopped(temp):
    if temp == Hot():
        Warm()
    else:
        Cold()


@inline
def Hot():
    2


@inline
def Warm():
    1


@inline
def Cold():
    0


In [3]:
gas_and_power = {
    'name': 'quantdsl.priceprocess.blackscholes.BlackScholesPriceProcess',
    'market': ['GAS', 'POWER'],
    'sigma': [0.3, 0.3],
    'rho': [[1.0, 0.8], [0.8, 1.0]],
    'curve': {
            'GAS': [
                ('2012-1-1', 11.0),
                ('2012-1-2', 11.0),
                ('2012-1-3', 1.0),
                ('2012-1-4', 1.0),
                ('2012-1-5', 11.0),
            ],
            'POWER': [
                ('2012-1-1', 1.0),
                ('2012-1-2', 1.0),
                ('2012-1-3', 11.0),
                ('2012-1-4', 11.0),
                ('2012-1-5', 10.0),
            ]
        }
}

In [4]:
from quantdsl.interfaces.calcandplot import calc_print

results = calc_print(
    source_code=("from powerplant import PowerPlant, Cold, Date\n"
                 "PowerPlant(Date('2012-1-1'), Date('2012-1-5'), Cold())"),
    observation_date='2011-1-1',
    interest_rate=2.5,
    periodisation='daily',
    price_process=gas_and_power,
    verbose=True
)

Compiled 16 nodes 
Compilation in 0.150s
Simulation in 0.112s
Starting 112 node evaluations, please wait...
19/112 15.58% complete 81.33 eval/s running 1s eta 1ss

/Users/john/.virtualenvs/quantdsl-py36/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


112/112 100.00% complete 49.19 eval/s running 3s eta 0s
Evaluation in 2.281s


2012-01-02 POWER
Price:     1.00
Delta:     0.01
Hedge:    -0.01 ± 0.00
Cash:      0.02 ± 0.01

2012-01-02 GAS
Price:    10.97
Delta:    -0.04
Hedge:     0.04 ± 0.00
Cash:     -0.31 ± 0.07

2012-01-03 POWER
Price:    10.98
Delta:     0.32
Hedge:    -0.33 ± 0.00
Cash:      3.56 ± 0.08

2012-01-03 GAS
Price:     1.00
Delta:    -0.98
Hedge:     1.00 ± 0.00
Cash:     -0.97 ± 0.01

2012-01-04 POWER
Price:    10.98
Delta:     0.97
Hedge:    -1.00 ± 0.00
Cash:     10.71 ± 0.07

2012-01-04 GAS
Price:     1.00
Delta:    -0.98
Hedge:     1.00 ± 0.00
Cash:     -0.97 ± 0.01

2012-01-05 POWER
Price:     9.99
Delta:     0.31
Hedge:    -0.32 ± 0.00
Cash:      3.43 ± 0.12

2012-01-05 GAS
Price:    10.97
Delta:    -0.31
Hedge:     0.32 ± 0.00
Cash:     -3.05 ± 0.10

Net hedge GAS:       2.36 ± 0.02
Net hedge POWER:    -1.66 ± 0.02
Net hedge cash:     12.40 ± 0.10

Fair value: 12.39 ± 0.09
